In this notebook, we will try to integrate the **Maximum Subgroup Discrepancy** tool from `humancompatible.detect` into our **fairness-constrained training** routine.

**MSD** automatically detects groups (as defined by arbitrary combinations of protected attribute values) that are predicted at a higher or lower rate than their complement.

Our idea is to periodically detect new biases during the model training process, and dynamically add fairness constraints based on detected discrepancies.

In [1]:
# load and prepare data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import numpy as np
from folktables import ACSDataSource, ACSIncome, generate_categories

# load folktables data
data_source = ACSDataSource(survey_year="2018", horizon="1-Year", survey="person")
acs_data = data_source.get_data(states=["CA"], download=True)
definition_df = data_source.get_definitions(download=True)
categories = generate_categories(
    features=ACSIncome.features, definition_df=definition_df
)
df_feat, df_labels, _ = ACSIncome.df_to_pandas(
    acs_data, categories=categories, dummies=True
)

sens_feature_names = [
    "SCHL",
    "SEX",
    "RAC1P",
    "AGEP",
]  # leave OCCP out for now cause it has 524 (!) values
sens_col_names = [
    col for col in df_feat.columns if col.startswith(tuple(sens_feature_names))
]

features = df_feat.drop(columns=sens_col_names).to_numpy(dtype="float")
groups = df_feat[sens_col_names].to_numpy(dtype="float")
labels = df_labels.to_numpy(dtype="float")

# split
indices = np.arange(len(features))
(
    X_train,
    X_test,
    y_train,
    y_test,
    groups_train,
    groups_test,
    indices_train,
    indices_test,
) = train_test_split(features, labels, groups, indices, test_size=0.2, random_state=42)
# scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# make into a pytorch dataset, remove the sensitive attribute
features_train = torch.tensor(X_train, dtype=torch.float32)
labels_train = torch.tensor(y_train, dtype=torch.float32)
sens_train = torch.tensor(groups_train)
dataset_train = torch.utils.data.TensorDataset(features_train, labels_train)

---
---

As the constraint, we use `NormLoss` from `fairret`, which penalizes the model based on the ratio between the value of a statistic for each group and the overall value: $\sum_{s\in S}{|1-\frac{f(\theta, X_s, y_s)}{f(\theta, X, y)}|}$.

**Every N epochs (or until the loss stops decreasing?) run MSD and add a constraint for that pair of subgroups**

To run MSD efficiently, we need to get rid of the one-hot encoding, and do some minor transformations on the data. We will use a separate DataFrame for MSD inputs.

Issues: each time we add a constraint, we have to reevaluate sensitive attribute encodings.

In [43]:
from sklearn.preprocessing import OrdinalEncoder, KBinsDiscretizer

# for MSD, we need to do a bit of additional preprocessing:
# - add negations for binary features as a separate column (already done via categories+get_dummies)
# - discretize continuous features

df_feat_msd, df_labels, _ = ACSIncome.df_to_pandas(
    acs_data, categories=categories, dummies=True
)
df_feat_msd, df_labels = (
    df_feat_msd[sens_col_names].iloc[indices_train],
    df_labels.iloc[indices_train],
)

cont_sens_feature_names = ["AGEP"]
ds = KBinsDiscretizer(n_bins=10, encode="onehot-dense")
cont_bins = ds.fit_transform(df_feat_msd[cont_sens_feature_names])
for fidx, cont_feature in enumerate(cont_sens_feature_names):
    for i in range(ds.n_bins_[fidx]):
        lb, ub = ds.bin_edges_[fidx][i], ds.bin_edges_[fidx][i + 1]
        df_feat_msd["_".join([cont_feature, str(lb), str(ub)])] = cont_bins[
            :, i
        ].astype(bool)

df_feat_msd.drop(columns=cont_sens_feature_names, inplace=True)
df_feat_msd = df_feat_msd.convert_dtypes(convert_boolean=True)

Conjuncts_kwargs = {"solver": "glpk", "time_limit": 600}

# cat_features = list(set(df_feat_msd.columns) - set(cont_feature_names))
# FEATURE_PROCESSING = {
#     "POBP": lambda x: int(x) // 100,  # group by continents + US
#     "OCCP": lambda x: int(x) // 100,
#     "PUMA": lambda x: int(x) // 100,
#     "POWPUMA": lambda x: int(x) // 1000,
# }
# oe = OrdinalEncoder()
# df_feat_msd[cat_features] = oe.fit_transform(df_feat_msd[cat_features])

MSD_kwargs = {
    "n_samples": 800,
}

df_feat_msd

/home/andre/anaconda3/envs/detect-train/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


,"SCHL_1 or more years of college credit, no degree",SCHL_12th grade - no diploma,SCHL_Associate's degree,SCHL_Bachelor's degree,SCHL_Doctorate degree,SCHL_GED or alternative credential,SCHL_Grade 1,SCHL_Grade 10,SCHL_Grade 11,SCHL_Grade 2,...,AGEP_17.0_23.0,AGEP_23.0_28.0,AGEP_28.0_32.0,AGEP_32.0_37.0,AGEP_37.0_42.0,AGEP_42.0_47.0,AGEP_47.0_52.0,AGEP_52.0_57.0,AGEP_57.0_63.0,AGEP_63.0_94.0
82350,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
23973,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
92330,False,False,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
115743,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
82575,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,False,False,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
103694,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
131932,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
146867,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [44]:
from fairret.statistic import PositiveRate
from fairret.loss import NormLoss

dataset = torch.utils.data.TensorDataset(
    features_train, torch.tensor(df_feat_msd.to_numpy(dtype=bool)), labels_train
)

# we need bigger batch size to make sure that each batch contains members of each potential subgroup
dataloader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=128)

criterion = torch.nn.BCEWithLogitsLoss()

# MSD detects discrepancies in positive predictions
statistic = PositiveRate()
fair_criterion = NormLoss(statistic=statistic)
fair_crit_bound = 0.2

In [45]:
from torch.nn import Sequential

hsize1 = 32
hsize2 = 16
model_con = Sequential(
    torch.nn.Linear(features_train.shape[1], hsize1),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize1, hsize2),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize2, 1),
)

from torch.optim import Adam
from humancompatible.train.dual_optim import ALM

optimizer = Adam(model_con.parameters())
dual = ALM(m=1, momentum=0.2, dampening=0.2)

# add slack variables
slack_vars = torch.zeros(10, requires_grad=True)
optimizer.add_param_group(param_group={"params": slack_vars, "name": "slack"})

epochs = 100
msd_interval = 10

In [46]:
from humancompatible.detect import detect_and_score
from humancompatible.detect.helpers import report_subgroup_bias
from humancompatible.detect.methods.msd import get_conjuncts_MSD
import random
from tqdm import tqdm

constraints = []


# TODO: train until val learning rate plateaus
# TODO: automatically relax constraints if too harsh?

for epoch in range(epochs):
    # clear epoch stats
    loss_log = []
    c_log = []
    duals_log = []
    if (epoch % msd_interval == 0 and len(constraints) <= 5):
            ### find new MSD ###
            with torch.inference_mode():
                indices = random.sample(range(len(df_feat_msd)), MSD_kwargs["n_samples"])
                MSD_sample_model_input = dataset[indices][0]
                MSD_sample_x = df_feat_msd.iloc[indices].to_numpy(dtype=bool)
                # get binary predictions on sample
                # TODO: replace this with e.g. indicator of a true positive or whatever
                MSD_sample_y = (
                    torch.nn.functional.sigmoid(model_con(MSD_sample_model_input)).squeeze()
                    > 0.5
                ).numpy()
            
            if not np.any(MSD_sample_y):
                continue

            conj_indices = get_conjuncts_MSD(MSD_sample_x, MSD_sample_y, **Conjuncts_kwargs)
            print(f"Adding new rule: {df_feat_msd.columns[conj_indices].to_list()}")
            if conj_indices in constraints:
                continue
            
            constraints.append(conj_indices)
            if epoch > 0:
                dual.add_constraint_group(m=1)


    for batch_input, batch_sens, batch_label in dataloader:# tqdm(dataloader, leave=False):
        # calculate constraints and constraint grads
        out = model_con(batch_input)
        c_log.append([])
        batch_cons = []
        for j, conj_indices in enumerate(constraints):
            # one-hot evaluation of rule `j`
            sens_attrs = batch_sens[:, conj_indices]
            rule_eval = torch.prod(sens_attrs, dim=1, keepdim=True, dtype=bool)
            batch_rule_groups = torch.cat([rule_eval, ~rule_eval], dim=1)
            # evaluate fairret loss
            fair_loss = fair_criterion(out, batch_rule_groups)
            batch_cons.append(fair_loss.unsqueeze(0))
            c_log[-1].append([fair_loss.detach().item()])

        batch_cons = torch.cat(batch_cons) - fair_crit_bound + slack_vars[:len(batch_cons)]

        # primal eval and update
        loss = criterion(out, batch_label)
        lagrangian = dual.forward_update(loss, batch_cons)
        lagrangian.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_log.append(loss.detach().numpy())
        duals_log.append(dual.duals.detach().numpy())
        
        # slack variables must be non-negative. this is the "projection" step from the SSL-ALM paper
        with torch.no_grad():
            for s in slack_vars:
                if s < 0:
                    s.zero_()


    print(
        f"Epoch: {epoch}, "
        f"loss: {np.mean(loss_log[:-100])}, "
        f"constraints: {np.mean(c_log[:-100], axis=0)}, "
        f"dual: {np.mean(duals_log[:-100], axis=0)}"
    )


Adding new rule: ["SCHL_Associate's degree", 'SEX_Male']
Epoch: 0, loss: 0.46826639771461487, constraints: [[0.2205522]], dual: [0.22095282]
Epoch: 1, loss: 0.46371814608573914, constraints: [[0.20312997]], dual: [0.43269038]
Epoch: 2, loss: 0.45843788981437683, constraints: [[0.20374881]], dual: [0.43759423]
Epoch: 3, loss: 0.4517832100391388, constraints: [[0.19789998]], dual: [0.42269498]
Epoch: 4, loss: 0.45692679286003113, constraints: [[0.19773155]], dual: [0.44203988]
Epoch: 5, loss: 0.45174553990364075, constraints: [[0.20164972]], dual: [0.42802367]
Epoch: 6, loss: 0.45107316970825195, constraints: [[0.19874487]], dual: [0.42950717]
Epoch: 7, loss: 0.44518139958381653, constraints: [[0.1968969]], dual: [0.39155674]
Epoch: 8, loss: 0.4483039379119873, constraints: [[0.19979302]], dual: [0.4174498]
Epoch: 9, loss: 0.4449792802333832, constraints: [[0.20392439]], dual: [0.4071264]
Adding new rule: ["SCHL_Bachelor's degree"]
Epoch: 10, loss: 0.4741983115673065, constraints: [[0.16

KeyboardInterrupt: 

In [ ]:
batch_sens[:, 0].shape

torch.Size([96])

In [ ]:
from humancompatible.detect.binarizer import Bin


def rule_to_indicator(rule, features):
    """
    Given a rule and a tensor of one-hot variables, creates a new indicator column denoting the rule.
    `features`must have shape (`batch_size`,`n_features`)
    """
    for feature_idx, bin in rule:
        features[feature_idx]

In [ ]:
with torch.no_grad():
    f = model_con.forward(dataset[:][0])

In [ ]:
torch.mean(((torch.nn.functional.sigmoid(f) > 0.5) == dataset[:][2]).to(torch.float16))

tensor(0.6255, dtype=torch.float16)

In [ ]:
from fairret.statistic import PositiveRate

preds = torch.nn.functional.sigmoid(model_con(features_train))
pr = PositiveRate()
pr(preds, sens_train)

tensor([0.4769, 0.4559, 0.4392, 0.4757, 0.4919, 0.5193, 0.4537, 0.4199, 0.4385,
        0.4023, 0.3916, 0.4016, 0.4039, 0.4031, 0.4086, 0.4025, 0.4282, 0.4300,
        0.4723, 0.5128, 0.4259, 0.4359, 0.5164, 0.4432, 0.4597, 0.4572, 0.4796,
        0.5135, 0.4557, 0.4454, 0.4731, 0.4720, 0.4660, 0.4348, 0.4614, 0.4749],
       dtype=torch.float64, grad_fn=<IndexPutBackward0>)

In [ ]:
fair_criterion(model_con(features_train), sens_train)

tensor(2.4357, dtype=torch.float64, grad_fn=<SumBackward0>)

In [ ]:
from matplotlib import pyplot as plt

window_len = 5
c_mavg = np.array(
    [np.mean(ep_c_log[i : i + window_len]) for i in range(0, len(ep_c_log), window_len)]
)
plt.plot(np.array(c_mavg).flatten(), lw=0.05)
plt.hlines(y=fair_crit_bound, xmin=0, xmax=len(c_mavg), colors="black", ls="--")